## Imports

In [ ]:
import Config
from Signals import *
import Portfolio
import Get_Data
import Process_Data
import Dashboard
import Backtest

## Get Data

In [ ]:
# Re-actualisation du data
#Get_Data.get_yahoo_finance_data(Config.yahoo_assets, Config.FILE_PATH_YF)
#Get_Data.clean_and_process_prices(Config.FILE_PATH_YF)

data_prices_df, assets_names = Get_Data.load_prices_from_csv(Config.FILE_PATH_YF)

## Config

In [ ]:
indicators_and_params, assets_to_backtest = Config.dynamic_config(assets_names, auto=True)

## Process Data

In [ ]:
# Process Data
data_prices_df = Process_Data.adjust_prices_for_inversion(data_prices_df, ['VIXY'])

data = Process_Data.process_category_data(assets_names,
                                          data_prices_df, 
                                          assets_to_backtest,
                                          initial_equity=100)   

prices_array_test = data['returnstreams']['prices_array']
volatility_adjusted_pct_returns_array_test = data['returnstreams']['volatility_adjusted_pct_returns']
log_returns_array_test = data['returnstreams']['log_returns_array']
asset_names_test = data['returnstreams']['asset_names']
dates_index = data['returnstreams']['dates']

In [ ]:
# Suppression des objets désormais inutiles
del (
    assets_to_backtest,
    data_prices_df, 
    assets_names, 
    data,
    )

## Backtest

In [ ]:
# Trading Signals
raw_adjusted_returns_df = Backtest.process_backtest(
                                                    prices_array_test,
                                                    log_returns_array_test,
                                                    volatility_adjusted_pct_returns_array_test,
                                                    asset_names_test,
                                                    dates_index,
                                                    indicators_and_params)

In [ ]:
# Suppression des objets désormais inutiles
del (
    prices_array_test,
    log_returns_array_test,
    volatility_adjusted_pct_returns_array_test,
    dates_index,
    )

## Portfolio Construction

In [ ]:
#portfolio_base_structure = Portfolio.classify_assets(asset_names_test, Config.portfolio_etf)

#relativized_sharpes_df = Portfolio.relative_sharpe_on_confidence_period(raw_adjusted_returns_df, sharpe_lookback = 5000, confidence_lookback=2500)
#optimized_returns_rltv = raw_adjusted_returns_df * relativized_sharpes_df.shift(1)
#test_strategy_returns = Portfolio.calculate_daily_average_returns(optimized_returns_rltv, by_method=True, by_class=True, by_asset=True)
#test_asset_returns = Portfolio.generate_recursive_strategy_means(test_strategy_returns, Config.portfolio_strategies)
#test_global_returns = Portfolio.generate_recursive_means(test_asset_returns, portfolio_base_structure)
#test_global_returns = test_global_returns.rename(columns={test_global_returns.columns[0]: 'cluster_optimized'})

equal_weights_asset_returns = Portfolio.calculate_daily_average_returns(raw_adjusted_returns_df, by_asset=True)
equal_weights_global_returns = Portfolio.calculate_daily_average_returns(equal_weights_asset_returns, global_avg=True)
equal_weights_global_returns = equal_weights_global_returns.rename(columns={equal_weights_global_returns.columns[0]: 'equal_weights'})

# Concaténation des DataFrames
#total_df = pd.concat([test_global_returns, equal_weights_global_returns], axis=1).dropna()

## Visualization

In [ ]:
# Visualization
test_returns = equal_weights_global_returns#.loc['2015-01-01':]

original_equity_curves = Process_Data.equity_curves_calculs(test_returns, initial_equity = 100000)

Dashboard.equity(original_equity_curves)
#Dashboard.plot_final_equity_values(test_returns)
#Dashboard.drawdowns(original_equity_curves)
#Dashboard.max_drawdowns(test_returns)
#Dashboard.volatility(test_returns, means=False)
Dashboard.rolling_sharpe_ratio(test_returns, window_size=1250)
sharpe_ratios = Dashboard.overall_sharpe_ratios(test_returns)
#Dashboard.overall_monthly_skew(test_returns)
#Dashboard.sharpe_ratios_heatmap(raw_adjusted_returns_df, 'LenVol', 'LenSmooth')
#Dashboard.sharpe_ratios_3d_scatter_plot(raw_adjusted_returns_df, ['LenST', 'LenLT', 'MacdLength'])
#Dashboard.params_relative_impact_on_sharpe(daily_returns, ['LenST', 'LenLT'])
#Dashboard.sharpe_ratios_yearly_table(combined_df)
#Dashboard.correlation_heatmap(test_returns)
#Dashboard.average_correlation_bar_chart(test_returns)
#Dashboard.sharpe_correlation_ratio_bar_chart(daily_returns)
#Dashboard.returns_distribution(daily_returns)
#Dashboard.plot_static_clusters(test, 4, 1, 1)
#clusters_dict
#sharpe_ratios
#nan_dict